# 📄 Script de Extração e Processamento de Dados do DataJud

Este notebook tem como objetivo **automatizar a coleta, o processamento e o armazenamento de metadados judiciais** extraídos da API do [DataJud](https://www.cnj.jus.br/sistemas/datajud/) para processos previamente identificados.

## 🔧 Funcionalidades Principais

- 🔍 **Consulta de processos** com `qtd_tokens_ementa > 100` que ainda não foram sumarizados (`response IS NULL`).
- 🌐 **Identificação automática dos endpoints corretos** da API do DataJud com base no tribunal de origem do processo.
- 🤖 **Geração automática de resposta estruturada** (em JSON) utilizando o modelo LLM `pixtral-12b` via API.
- 🧠 **Extração inteligente dos campos** da ementa usando técnicas de NLP e regex para garantir formatação consistente.
- 💾 **Atualização direta no banco de dados PostgreSQL** com as respostas geradas, processo por processo.
- 🔁 **Loop automático de execução a cada 10 minutos**, permitindo execução contínua para ambientes de produção.

## ⏱️ Ciclo de Execução

1. Aguarda 10 minutos (`time.sleep(600)`)
2. Executa `extrair()`
3. Volta para o passo 1 (loop infinito)

---

## 📂 Tabelas Envolvidas

- `processos`: contém os dados brutos das ementas e metadados dos processos.
- `metadados_datajud`: armazena os dados extraídos da API do DataJud.

## ⚠️ Observações

- O campo `numero_processo_limpo` já deve estar padronizado para consulta direta na API.
- Os endpoints da API são diferentes para cada tribunal e devem estar previamente definidos na coluna `endpoint`.
- Caso ocorra um erro durante a consulta ou inserção, o sistema aguarda **5 minutos** antes de tentar continuar.

## 👨‍💻 Desenvolvido para:

Análise automatizada de decisões judiciais com foco em jurimetria e políticas públicas, integrando grandes volumes de dados com modelos de linguagem para geração estruturada de conhecimento.



In [8]:
import re
import psycopg2
import requests
import json
from IPython.display import clear_output
import unicodedata



In [9]:
# Dicionário com as siglas dos tribunais e seus respectivos endpoints
endpoints = {
    "TST": "https://api-publica.datajud.cnj.jus.br/api_publica_tst/_search",
    "TSE": "https://api-publica.datajud.cnj.jus.br/api_publica_tse/_search",
    "STJ": "https://api-publica.datajud.cnj.jus.br/api_publica_stj/_search",
    "STM": "https://api-publica.datajud.cnj.jus.br/api_publica_stm/_search",
    "TRF-1": "https://api-publica.datajud.cnj.jus.br/api_publica_trf1/_search",
    "TRF-2": "https://api-publica.datajud.cnj.jus.br/api_publica_trf2/_search",
    "TRF-3": "https://api-publica.datajud.cnj.jus.br/api_publica_trf3/_search",
    "TRF-4": "https://api-publica.datajud.cnj.jus.br/api_publica_trf4/_search",
    "TRF-5": "https://api-publica.datajud.cnj.jus.br/api_publica_trf5/_search",
    "TRF-6": "https://api-publica.datajud.cnj.jus.br/api_publica_trf6/_search",
    "TJ-AC": "https://api-publica.datajud.cnj.jus.br/api_publica_tjac/_search",
    "TJ-AL": "https://api-publica.datajud.cnj.jus.br/api_publica_tjal/_search",
    "TJ-AM": "https://api-publica.datajud.cnj.jus.br/api_publica_tjam/_search",
    "TJ-AP": "https://api-publica.datajud.cnj.jus.br/api_publica_tjap/_search",
    "TJ-BA": "https://api-publica.datajud.cnj.jus.br/api_publica_tjba/_search",
    "TJ-CE": "https://api-publica.datajud.cnj.jus.br/api_publica_tjce/_search",
    "TJ-DF": "https://api-publica.datajud.cnj.jus.br/api_publica_tjdft/_search",
    "TJ-ES": "https://api-publica.datajud.cnj.jus.br/api_publica_tjes/_search",
    "TJ-GO": "https://api-publica.datajud.cnj.jus.br/api_publica_tjgo/_search",
    "TJ-MA": "https://api-publica.datajud.cnj.jus.br/api_publica_tjma/_search",
    "TJ-MG": "https://api-publica.datajud.cnj.jus.br/api_publica_tjmg/_search",
    "TJ-MS": "https://api-publica.datajud.cnj.jus.br/api_publica_tjms/_search",
    "TJ-MT": "https://api-publica.datajud.cnj.jus.br/api_publica_tjmt/_search",
    "TJ-PA": "https://api-publica.datajud.cnj.jus.br/api_publica_tjpa/_search",
    "TJ-PB": "https://api-publica.datajud.cnj.jus.br/api_publica_tjpb/_search",
    "TJ-PE": "https://api-publica.datajud.cnj.jus.br/api_publica_tjpe/_search",
    "TJ-PI": "https://api-publica.datajud.cnj.jus.br/api_publica_tjpi/_search",
    "TJ-PR": "https://api-publica.datajud.cnj.jus.br/api_publica_tjpr/_search",
    "TJ-RJ": "https://api-publica.datajud.cnj.jus.br/api_publica_tjrj/_search",
    "TJ-RN": "https://api-publica.datajud.cnj.jus.br/api_publica_tjrn/_search",
    "TJ-RO": "https://api-publica.datajud.cnj.jus.br/api_publica_tjro/_search",
    "TJ-RR": "https://api-publica.datajud.cnj.jus.br/api_publica_tjrr/_search",
    "TJ-RS": "https://api-publica.datajud.cnj.jus.br/api_publica_tjrs/_search",
    "TJ-SC": "https://api-publica.datajud.cnj.jus.br/api_publica_tjsc/_search",
    "TJ-SE": "https://api-publica.datajud.cnj.jus.br/api_publica_tjse/_search",
    "TJ-SP": "https://api-publica.datajud.cnj.jus.br/api_publica_tjsp/_search",
    "TJ-TO": "https://api-publica.datajud.cnj.jus.br/api_publica_tjto/_search",
    "TRT-1": "https://api-publica.datajud.cnj.jus.br/api_publica_trt1/_search",
    "TRT-2": "https://api-publica.datajud.cnj.jus.br/api_publica_trt2/_search",
    "TRT-3": "https://api-publica.datajud.cnj.jus.br/api_publica_trt3/_search",
    "TRT-4": "https://api-publica.datajud.cnj.jus.br/api_publica_trt4/_search",
    "TRT-5": "https://api-publica.datajud.cnj.jus.br/api_publica_trt5/_search",
    "TRT-6": "https://api-publica.datajud.cnj.jus.br/api_publica_trt6/_search",
    "TRT-7": "https://api-publica.datajud.cnj.jus.br/api_publica_trt7/_search",
    "TRT-8": "https://api-publica.datajud.cnj.jus.br/api_publica_trt8/_search",
    "TRT-9": "https://api-publica.datajud.cnj.jus.br/api_publica_trt9/_search",
    "TRT-10": "https://api-publica.datajud.cnj.jus.br/api_publica_trt10/_search",
    "TRT-11": "https://api-publica.datajud.cnj.jus.br/api_publica_trt11/_search",
    "TRT-12": "https://api-publica.datajud.cnj.jus.br/api_publica_trt12/_search",
    "TRT-13": "https://api-publica.datajud.cnj.jus.br/api_publica_trt13/_search",
    "TRT-14": "https://api-publica.datajud.cnj.jus.br/api_publica_trt14/_search",
    "TRT-15": "https://api-publica.datajud.cnj.jus.br/api_publica_trt15/_search",
    "TRT-16": "https://api-publica.datajud.cnj.jus.br/api_publica_trt16/_search",
    "TRT-17": "https://api-publica.datajud.cnj.jus.br/api_publica_trt17/_search",
    "TRT-18": "https://api-publica.datajud.cnj.jus.br/api_publica_trt18/_search",
    "TRT-19": "https://api-publica.datajud.cnj.jus.br/api_publica_trt19/_search",
    "TRT-20": "https://api-publica.datajud.cnj.jus.br/api_publica_trt20/_search",
    "TRT-21": "https://api-publica.datajud.cnj.jus.br/api_publica_trt21/_search",
    "TRT-22": "https://api-publica.datajud.cnj.jus.br/api_publica_trt22/_search",
    "TRT-23": "https://api-publica.datajud.cnj.jus.br/api_publica_trt23/_search",
    "TRT-24": "https://api-publica.datajud.cnj.jus.br/api_publica_trt24/_search",
    "TNU": "https://api-publica.datajud.cnj.jus.br/api_publica_tnu/_search",
    "TRE-PR": "https://api-publica.datajud.cnj.jus.br/api_publica_tre-pr/_search",
    "TRE-RN": "https://api-publica.datajud.cnj.jus.br/api_publica_tre-rn/_search",
    "TRE-MG": "https://api-publica.datajud.cnj.jus.br/api_publica_tre-mg/_search",
    "TRE-ES": "https://api-publica.datajud.cnj.jus.br/api_publica_tre-es/_search",
    "TRE-PB": "https://api-publica.datajud.cnj.jus.br/api_publica_tre-pb/_search",
    "TRE-SP": "https://api-publica.datajud.cnj.jus.br/api_publica_tre-sp/_search",
    "TRE-CE": "https://api-publica.datajud.cnj.jus.br/api_publica_tre-ce/_search",
    "TRE-GO": "https://api-publica.datajud.cnj.jus.br/api_publica_tre-go/_search"
}


def limpar_tribunal(texto):
    if not texto:
        return ''
    texto = texto.strip()
    texto = ''.join(c for c in texto if unicodedata.category(c)[0] != 'C')  # remove invisíveis
    return texto




OBTENDO O ENDEREÇO DOS ENDPOINTS E ARMAZENANDO NA TABELA PARA CONSULTA POSTERIOR

In [21]:
import psycopg2
import unicodedata

# Função para limpar string do tribunal
def limpar_tribunal(texto):
    if not texto:
        return ''
    texto = texto.strip()
    texto = ''.join(c for c in texto if unicodedata.category(c)[0] != 'C')  # remove caracteres de controle
    return texto

# Conexão com o banco
conn = psycopg2.connect(
    host="localhost",
    database="PROCESSOS",
    user="postgres",
    password="admin",
    port="5432"
)
conn.autocommit = True
cur = conn.cursor()

# Selecionar processos sem endpoint atribuído
cur.execute("""
    SELECT id, numero_processo_limpo, numero_processo_tribunal, tribunal 
    FROM processos 
    WHERE endpoint IS NULL AND qtd_tokens_ementa > 100
""")
processos = cur.fetchall()

# Atualização por processo
for id, numero_processo_limpo, numero_processo_tribunal, tribunal in processos:
    tribunal = limpar_tribunal(tribunal)
    endpoint = endpoints.get(tribunal, None)

    if endpoint:
        cur.execute("""
            UPDATE public.processos 
            SET endpoint = %s 
            WHERE numero_processo_tribunal = %s
        """, (endpoint, numero_processo_tribunal))
        print(f"[OK] Processo {numero_processo_tribunal} ({numero_processo_limpo}) - {tribunal} => {endpoint}")
    else:
        #print(tribunal)
        print(f"[ERRO] Tribunal '{repr(tribunal)}' não encontrado para o processo {numero_processo_tribunal}")
    clear_output(wait=True) 
# Encerrar conexões
cur.close()
conn.close()
print("✅ Atualização concluída.")


✅ Atualização concluída.


Alguns tribunais não tem endpoint:

select count(tribunal), tribunal from processos where endpoint is null and qtd_tokens_ementa > 100
group by tribunal order by countTribunal qtd processos

count	tribunal
1	CNJ
1	TRE-MS
2	TRE-AM
2	TRE-MA
2	TRE-RO
3	TAT-MS
3	TRE-SE
5	TRE-BA
5	TRE-PE
5	TCE-PR
5	TRE-PI
6	TRE-MT
7	TRE-SC
8	TRE-RJ
12	TRE-RS
13	STF
15	TCE-PE
23	TCE-MS
51	TCE-MG
56	CARF
119	TCU

344 processos são desses tribunais


In [ ]:
#Consultando cada processo no datajud

In [18]:

# Função para processar e inserir os dados no banco
def processa_e_insere_dados(response, numero_processo_tribunal, numero_processo, cursor, conn):
    if response.status_code == 200:
        try:
            json_response = response.json()
        except json.JSONDecodeError:
            print(f"Resposta inválida em JSON para o processo {numero_processo}")
            return

        if json_response.get('hits', {}).get('total', {}).get('value', 0) > 0:
            dados = json_response['hits']['hits'][0]['_source']

            # Extraindo informações
            codigo_municipio = dados.get('orgaoJulgador', {}).get('codigoMunicipioIBGE')
            tribunal = dados.get('tribunal')
            data_hora_atualizacao = dados.get('dataHoraUltimaAtualizacao')
            grau = dados.get('grau')
            data_ajuizamento = dados.get('dataAjuizamento')
            nivel_sigilo = dados.get('nivelSigilo')
            formato_codigo = dados.get('formato', {}).get('codigo')
            formato_nome = dados.get('formato', {}).get('nome')
            sistema_codigo = dados.get('sistema', {}).get('codigo')
            sistema_nome = dados.get('sistema', {}).get('nome')
            orgao_julgador = json.dumps(dados.get('orgaoJulgador', {}))
            movimentos = json.dumps(dados.get('movimentos', []))

            # Assuntos
            assunto_codigo = assunto_nome = segundo_assunto_codigo = segundo_assunto_nome = None
            try:
                assuntos = dados.get('assuntos', [])
                if len(assuntos) > 0:
                    assunto_codigo = assuntos[0].get('codigo')
                    assunto_nome = assuntos[0].get('nome')
                if len(assuntos) > 1:
                    segundo_assunto_codigo = assuntos[1].get('codigo')
                    segundo_assunto_nome = assuntos[1].get('nome')
            except Exception as e:
                print(f"Erro ao processar 'assuntos' para o processo {numero_processo}: {e}")

            # Inserção no banco
            cursor.execute("""
                INSERT INTO public.metadados_datajud (
                    numeroprocesso, numero_processo_tribunal, classe_nome, codigomunicipioibge, tribunal, 
                    datahoraultimaatualizacao, grau, dataajuizamento, assunto_codigo, 
                    assunto_nome, nivelSigilo, formato_codigo, formato_nome, sistema_codigo, sistema_nome, 
                    orgaoJulgador, movimentos, assunto_codigo_2, assunto_nome_2
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, (
                dados.get('numeroProcesso'),
                numero_processo_tribunal,
                dados.get('classe', {}).get('nome'),
                codigo_municipio,
                tribunal,
                data_hora_atualizacao,
                grau,
                data_ajuizamento,
                assunto_codigo,
                assunto_nome,
                nivel_sigilo,
                formato_codigo,
                formato_nome,
                sistema_codigo,
                sistema_nome,
                orgao_julgador,
                movimentos,
                segundo_assunto_codigo,
                segundo_assunto_nome
            ))
            conn.commit()
            print(f"✔️ Dados inseridos para o processo: {numero_processo}")
        else:
            print(f"⚠️ Nenhum dado encontrado para o processo: {numero_processo}")
    else:
        print(f"❌ Erro ao consultar o processo {numero_processo}: {response.status_code} - {response.text}")



In [39]:
def consulta():
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="PROCESSOS",
            user="postgres",
            password="admin",
            port="5432"
        )
        cur = conn.cursor()
        contador = 0

        cur.execute('''
            SELECT proc.numero_processo_limpo, proc.tribunal, proc.numero_processo_tribunal, proc.endpoint
            FROM public.processos proc
            WHERE 
                proc.endpoint IS NOT NULL 
                AND proc.qtd_tokens_ementa > 100
                AND NOT EXISTS (
                    SELECT 1 
                    FROM public.metadados_datajud meta
                    WHERE meta.numero_processo_tribunal = proc.numero_processo_tribunal
                );
        ''')


        processos = cur.fetchall()

        for processo in processos:
            contador += 1
            numero_processo_limpo = processo[0]
            tribunal = processo[1]
            numero_processo_tribunal = processo[2]
            endpoint = processo[3]
            
            if not numero_processo_limpo or not endpoint:
                print(f"⚠️ Processo ou endpoint ausente: {numero_processo_limpo.zfill(20)}")
                continue

            payload = json.dumps({
                "query": {
                    "match": {
                        "numeroProcesso": numero_processo_limpo.zfill(20)
                    }
                }
            })

            headers = {
                'Authorization': 'APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==',
                'Content-Type': 'application/json'
            }

            response = requests.post(endpoint, headers=headers, data=payload)

            if response.status_code == 200:
                hits_total = response.json().get("hits", {}).get("total", {}).get("value", 0)
                if hits_total > 0:
                    print(f"✅ Processo {numero_processo_limpo} retornou {hits_total} resultado(s).")
                    processa_e_insere_dados(response, numero_processo_tribunal, numero_processo_limpo, cur, conn)
                    print(f"📦 Registro processado: {contador}")
                else:
                    print(f"⚠️ Nenhum dado encontrado para {numero_processo_limpo}.")
            else:
                print(f"❌ Erro na API para o processo {numero_processo_limpo}. Status: {response.status_code}")
            
            clear_output(wait=True) 

    finally:
        cur.close()
        conn.close()
        


In [40]:
consulta()

⚠️ Nenhum dado encontrado para 00005121820148050150.


#7367 processos estão com o numero duplicado 
#5017578302015404710850175783020154047108

SELECT COUNT(*) AS total_duplicados
FROM public.processos
WHERE LENGTH(numero_processo_limpo) % 2 = 0
  AND SUBSTRING(numero_processo_limpo FROM 1 FOR LENGTH(numero_processo_limpo)/2) =
      SUBSTRING(numero_processo_limpo FROM (LENGTH(numero_processo_limpo)/2)+1);
      
A consulta abaixo divide o numero deles e consulta

In [36]:
def dividir_numero_duplicado(numero):
    tamanho = len(numero)
    if tamanho % 2 == 0:
        metade = tamanho // 2
        if numero[:metade] == numero[metade:]:
            return numero[:metade]
    # Se não for duplicado, retorna o próprio número sem alteração
    return numero



def consulta_dividindo():
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="PROCESSOS",
            user="postgres",
            password="admin",
            port="5432"
        )
        cur = conn.cursor()
        contador = 0

        cur.execute('''
            SELECT proc.numero_processo_limpo, proc.tribunal, proc.numero_processo_tribunal, proc.endpoint
            FROM public.processos proc
            WHERE 
                proc.endpoint IS NOT NULL 
                AND proc.qtd_tokens_ementa > 100
                AND NOT EXISTS (
                    SELECT 1 
                    FROM public.metadados_datajud meta
                    WHERE meta.numero_processo_tribunal = proc.numero_processo_tribunal
                );
        ''')


        processos = cur.fetchall()

        for processo in processos:
            contador += 1
            numero_processo_limpo = processo[0]
            tribunal = processo[1]
            numero_processo_tribunal = processo[2]
            endpoint = processo[3]

            if not numero_processo_limpo or not endpoint:
                print(f"⚠️ Processo ou endpoint ausente: {numero_processo_limpo}")
                continue

            payload = json.dumps({
                "query": {
                    "match": {
                        "numeroProcesso": dividir_numero_duplicado(numero_processo_limpo)
                    }
                }
            })

            headers = {
                'Authorization': 'APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==',
                'Content-Type': 'application/json'
            }

            response = requests.post(endpoint, headers=headers, data=payload)

            if response.status_code == 200:
                hits_total = response.json().get("hits", {}).get("total", {}).get("value", 0)
                if hits_total > 0:
                    print(f"✅ Processo {numero_processo_limpo} retornou {hits_total} resultado(s).")
                    processa_e_insere_dados(response, numero_processo_tribunal, numero_processo_limpo, cur, conn)
                    print(f"📦 Registro processado: {contador}")
                else:
                    print(f"⚠️ Nenhum dado encontrado para {numero_processo_limpo}.")
            else:
                print(f"❌ Erro na API para o processo {numero_processo_limpo}. Status: {response.status_code}")
            
            clear_output(wait=True) 

    finally:
        cur.close()
        conn.close()

consulta_dividindo()

✅ Processo 0711915342017807000307119153420178070003 retornou 1 resultado(s).
✔️ Dados inseridos para o processo: 0711915342017807000307119153420178070003
📦 Registro processado: 14721


In [41]:
def consulta_apata_dois_ultimos():
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="PROCESSOS",
            user="postgres",
            password="admin",
            port="5432"
        )
        cur = conn.cursor()
        contador = 0

        cur.execute('''
            SELECT proc.numero_processo_limpo, proc.tribunal, proc.numero_processo_tribunal, proc.endpoint
            FROM public.processos proc
            WHERE 
                proc.endpoint IS NOT NULL 
                AND proc.qtd_tokens_ementa > 100
                AND NOT EXISTS (
                    SELECT 1 
                    FROM public.metadados_datajud meta
                    WHERE meta.numero_processo_tribunal = proc.numero_processo_tribunal
                );
        ''')

        processos = cur.fetchall()

        for processo in processos:
            contador += 1
            numero_processo_limpo = processo[0]
            tribunal = processo[1]
            numero_processo_tribunal = processo[2]
            endpoint = processo[3]

            if not numero_processo_limpo or not endpoint:
                print(f"⚠️ Processo ou endpoint ausente: {numero_processo_limpo}")
                continue

            # Ajuste no número do processo
            numero_ajustado = numero_processo_limpo.strip()
            if len(numero_ajustado) > 20:
                numero_ajustado = numero_ajustado[:-2]  # remove os dois últimos dígitos
            numero_ajustado = numero_ajustado.zfill(20)

            payload = json.dumps({
                "query": {
                    "match": {
                        "numeroProcesso": numero_ajustado
                    }
                }
            })

            headers = {
                'Authorization': 'APIKey cDZHYzlZa0JadVREZDJCendQbXY6SkJlTzNjLV9TRENyQk1RdnFKZGRQdw==',
                'Content-Type': 'application/json'
            }

            response = requests.post(endpoint, headers=headers, data=payload)

            if response.status_code == 200:
                hits_total = response.json().get("hits", {}).get("total", {}).get("value", 0)
                if hits_total > 0:
                    print(f"✅ Processo {numero_ajustado} retornou {hits_total} resultado(s).")
                    processa_e_insere_dados(response, numero_processo_tribunal, numero_ajustado, cur, conn)
                    print(f"📦 Registro processado: {contador}")
                else:
                    print(f"⚠️ Nenhum dado encontrado para {numero_ajustado}.")
            else:
                print(f"❌ Erro na API para o processo {numero_ajustado}. Status: {response.status_code}")

            clear_output(wait=True)

    finally:
        cur.close()
        conn.close()


In [43]:
consulta_apata_dois_ultimos()

⚠️ Nenhum dado encontrado para 00005121820148050150.
